In [1]:
import torch
import torch.nn as nn
import numpy as np


In [3]:
with open("shakespeare.txt", "r", encoding="utf-8") as f:
    text = f.read()

print("Total characters:", len(text))
print(text[:300])


Total characters: 976
From fairest creatures we desire increase,
That thereby beauty's rose might never die,
But as the riper should by time decease,
His tender heir might bear his memory:

To be, or not to be, that is the question:
Whether 'tis nobler in the mind to suffer
The slings and arrows of outrageous fortune,
Or


In [6]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

char_to_idx = {ch: i for i, ch in enumerate(chars)}
idx_to_char = {i: ch for i, ch in enumerate(chars)}

print("Vocabulary size:", vocab_size)


Vocabulary size: 47


In [7]:
seq_length = 40  # hyperparameter
inputs = []
targets = []

for i in range(len(text) - seq_length):
    inputs.append([char_to_idx[ch] for ch in text[i:i+seq_length]])
    targets.append(char_to_idx[text[i+seq_length]])

inputs = torch.tensor(inputs)
targets = torch.tensor(targets)
#seq_length: how much past context the LSTM sees

In [8]:
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out


In [9]:
embed_size = 64        # size of character embedding
hidden_size = 128      # LSTM memory size
learning_rate = 0.003

model = LSTMModel(vocab_size, embed_size, hidden_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [10]:
epochs = 5  # keep small for demo

for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")


Epoch [1/5], Loss: 3.8364
Epoch [2/5], Loss: 3.7680
Epoch [3/5], Loss: 3.6916
Epoch [4/5], Loss: 3.5913
Epoch [5/5], Loss: 3.4454


In [11]:
def generate_text(start_text, length=200):
    model.eval()
    result = start_text

    for _ in range(length):
        x = torch.tensor([[char_to_idx[ch] for ch in result[-seq_length:]]])
        out = model(x)
        char_idx = torch.argmax(out, dim=1).item()
        result += idx_to_char[char_idx]

    return result


In [12]:
print(generate_text("To be or not"))


To be or not e e a e a e a e a e a e a e a e a e a e a e a e a e a e a e a e a e a e a e a e a e a e a e a e a e a e a e a e a e a e a e a e a e a e a e a e a e a e a e a e a e a e a e a e a e a e a e a e a e a e


Hyperparameters are values that are chosen before training a neural network and control how the model learns. In this project, parameters such as sequence length, embedding size, hidden size, learning rate, and number of epochs were selected manually. These values affect the model’s learning speed, memory capacity, and overall performance.